In [1]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.nn import functional as F
#from torch.utils.data import Dataset
from datasets import Dataset

# 读取并预处理数据集
df = pd.read_csv('validation_labels.csv')
# 删除包含 NaN 的行
df = df.dropna()

df

2025-03-09 18:34:40.133373: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 18:34:40.145786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 18:34:40.160593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 18:34:40.164976: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 18:34:40.175782: I tensorflow/core/platform/cpu_feature_guar

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_37,x_38,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40
0,R1107_1,G,1,-5.499000,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
1,R1107_2,G,2,-5.826000,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2,R1107_3,G,3,-5.849000,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
3,R1107_4,G,4,-5.784000,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
4,R1107_5,G,5,-5.755000,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1190_114,U,114,87.870003,105.432999,115.183998,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2511,R1190_115,U,115,92.911003,105.394997,113.741997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2512,R1190_116,U,116,99.012001,105.749001,113.073997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2513,R1190_117,U,117,103.861000,103.453003,114.589996,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18


In [2]:
df['molecule_id'] = df['ID'].apply(lambda x: x.split('_')[0])
df['residue_id'] = df['resid']
df

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40,molecule_id,residue_id
0,R1107_1,G,1,-5.499000,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,1
1,R1107_2,G,2,-5.826000,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,2
2,R1107_3,G,3,-5.849000,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,3
3,R1107_4,G,4,-5.784000,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,4
4,R1107_5,G,5,-5.755000,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1190_114,U,114,87.870003,105.432999,115.183998,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,114
2511,R1190_115,U,115,92.911003,105.394997,113.741997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,115
2512,R1190_116,U,116,99.012001,105.749001,113.073997,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,116
2513,R1190_117,U,117,103.861000,103.453003,114.589996,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1190,117


In [3]:
groups = df.groupby('molecule_id')
data = []
for molecule_id, group in groups:
    group = group.sort_values('residue_id')
    sequence = ''.join(group['resname'].values)
    labels = group[['x_1', 'y_1', 'z_1']].values
    data.append((sequence, labels))

data[0]

('GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU',
 array([[ -5.49900007,   8.52000046,   8.60499954],
        [ -5.82600021,  10.45300007,  14.01000023],
        [ -5.84899998,  14.76799965,  17.58499908],
        [ -5.78399992,  19.98500061,  18.66600037],
        [ -5.75500011,  25.53300095,  17.13299942],
        [ -6.22700024,  30.09300041,  13.96500015],
        [ -9.01599979,  37.02999878,  11.30599976],
        [ -9.02600002,  31.55400085,   8.72500038],
        [-13.9119997 ,  30.90800095,   8.34700012],
        [-22.27300072,  33.25099945,   7.10500002],
        [-25.75200081,  28.85400009,   8.54800034],
        [-28.56699944,  25.02700043,   6.70900011],
        [-30.61300087,  22.20700073,   2.5999999 ],
        [-30.47400093,  20.33399963,  -2.32599998],
        [-27.7670002 ,  19.59399986,  -7.18900013],
        [-23.65099907,  18.54299927,  -9.92700005],
        [-18.34600067,  17.52599907, -10.07900047],
        [-14.49499989,  14.75599957,  -6.9

In [4]:
sequence, labels = data[0]
print(sequence)
print(labels)

GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU
[[ -5.49900007   8.52000046   8.60499954]
 [ -5.82600021  10.45300007  14.01000023]
 [ -5.84899998  14.76799965  17.58499908]
 [ -5.78399992  19.98500061  18.66600037]
 [ -5.75500011  25.53300095  17.13299942]
 [ -6.22700024  30.09300041  13.96500015]
 [ -9.01599979  37.02999878  11.30599976]
 [ -9.02600002  31.55400085   8.72500038]
 [-13.9119997   30.90800095   8.34700012]
 [-22.27300072  33.25099945   7.10500002]
 [-25.75200081  28.85400009   8.54800034]
 [-28.56699944  25.02700043   6.70900011]
 [-30.61300087  22.20700073   2.5999999 ]
 [-30.47400093  20.33399963  -2.32599998]
 [-27.7670002   19.59399986  -7.18900013]
 [-23.65099907  18.54299927  -9.92700005]
 [-18.34600067  17.52599907 -10.07900047]
 [-14.49499989  14.75599957  -6.94799995]
 [-12.22599983  11.53999996  -2.98799992]
 [-12.13300037   6.88399982  -0.685     ]
 [-13.11299992   2.07399988  -3.01900005]
 [-11.57299995  -3.81200004  -7.13100004]
 [-16.

In [5]:
# 准备输入和标签
def prepare_input_labels(sequence, labels):
    seq_pos_list_local = []
    for i in range(0, len(sequence)):
        seq = sequence[0:i+1].replace("U","T") #U-->T
        #如果seq长度大于256则只要最后面的256个字符
        if len(seq) > 1024:
            seq = seq[-1024:]
        pos = labels[i].tolist()
        #print(seq, pos)
        data = {"seq":seq, "label":pos}
        seq_pos_list_local.append(data)
    return seq_pos_list_local
        
#seq_pos_list = prepare_input_labels(sequence, labels)

In [6]:
seq_pos_list_train = []
for sequence, labels in data:
    seq_pos_list = prepare_input_labels(sequence, labels)
    seq_pos_list_train.extend(seq_pos_list)


In [7]:
seq_pos_list_train[0]

{'seq': 'G',
 'label': [-5.499000072479248, 8.520000457763672, 8.604999542236328]}

In [8]:
import json
filename = "rna_pos_1024_val_no_none.jsonl"
rna_file = open(filename,"w")
for item in seq_pos_list_train:
    rna_file.write(json.dumps(item) + "\n")

rna_file.close()